In [1]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import shutil
from PIL import Image
from PIL.ExifTags import TAGS
import re
import json

### Criar a pasta divida por ano / categoria e renomeia as fotos com data_hora_categoria

In [6]:
def create_year_and_category_folders(output_folder):
    """
    Cria as pastas de ano (2019 a 2025) e as pastas de categoria dentro de cada pasta de ano.
    """
    for year in range(2019, 2026):
        year_folder = os.path.join(output_folder, str(year))
        if not os.path.exists(year_folder):
            os.makedirs(year_folder)
            print(f"Created year folder: {year_folder}")


def extract_date_time_from_exif(image_path):
    """
    Tenta extrair a data e hora dos metadados EXIF da imagem.
    Retorna None se não encontrar a data e hora nos EXIF.
    """
    try:
        # Abre a imagem
        image = Image.open(image_path)
        exif_data = image._getexif()

        # Verifica se há dados EXIF
        if exif_data is not None:
            # Busca pelas tags relevantes ('DateTime', 'DateTimeOriginal', 'DateTimeDigitized')
            for tag, value in exif_data.items():
                if TAGS.get(tag) in ['DateTime', 'DateTimeOriginal', 'DateTimeDigitized']:
                    # O formato de data geralmente é 'YYYY:MM:DD HH:MM:SS'
                    date_time = value.split(' ')
                    date = date_time[0].replace(':', '_')
                    time = date_time[1].replace(':', '_')
                    return date, time
        else:
            print(f"Imagem sem EXIF: {image_path}")

    except Exception as e:
        print(f"Erro ao extrair EXIF de {image_path}: {e}")
    
    return None, None  # Retorna None se não encontrar a data e hora


def process_images(input_folder, output_folder):
    """
    Processa as imagens renomeando e organizando nas pastas de ano e categoria.
    """
    # Cria pastas de ano (2019 a 2025)
    create_year_and_category_folders(output_folder)

    # Percorre cada pasta de categoria dentro do diretório de entrada (input_folder)
    for category_folder in os.listdir(input_folder):
        category_folder_path = os.path.join(input_folder, category_folder)

        # Verifica se é uma pasta (categoria)
        if os.path.isdir(category_folder_path):
            print(f"Processando categoria: {category_folder}")

            # Percorre cada imagem dentro da pasta de categoria
            for filename in os.listdir(category_folder_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
                    # Extrai a data e hora da imagem pelos metadados EXIF
                    date, time = extract_date_time_from_exif(os.path.join(category_folder_path, filename))
                    
                    if not date:
                        print(f"Pulando '{filename}' porque não foi encontrada a data.")
                        continue  # Se não encontrar a data, pula essa imagem

                    year = date.split('_')[0]

                    # Cria a pasta de categoria dentro da pasta de ano
                    year_folder = os.path.join(output_folder, str(year))
                    category_output_folder = os.path.join(year_folder, category_folder)

                    # Verifica se a pasta de categoria existe dentro do ano, caso contrário, cria
                    if not os.path.exists(category_output_folder):
                        os.makedirs(category_output_folder)
                        print(f"Created category folder: {category_output_folder}")

                    # Renomeia a imagem para o formato 'ano_mes_dia_hora_categoria'
                    new_filename = f"{date}_{time}{os.path.splitext(filename)[1]}"

                    # Caminhos completos para a imagem original e para a nova imagem
                    original_image_path = os.path.join(category_folder_path, filename)
                    new_image_path = os.path.join(category_output_folder, new_filename)
                    
                    # Move e renomeia a imagem
                    shutil.move(original_image_path, new_image_path)
                    print(f"Movido e renomeado '{filename}' para '{new_filename}' em '{category_output_folder}'")

    print("Processamento completo.")

In [7]:
input_folder = "imagens_rita"
output_folder = "imagens_mao"

In [8]:
process_images(input_folder, output_folder)

Created year folder: imagens_mao\2019
Created year folder: imagens_mao\2020
Created year folder: imagens_mao\2021
Created year folder: imagens_mao\2022
Created year folder: imagens_mao\2023
Created year folder: imagens_mao\2024
Created year folder: imagens_mao\2025
Processando categoria: animais
Created category folder: imagens_mao\2023\animais
Movido e renomeado '2023_0405.JPEG' para '2023_07_11_17_01_20.JPEG' em 'imagens_mao\2023\animais'
Processando categoria: coisas da terra
Processando categoria: feridas
Created category folder: imagens_mao\2020\feridas
Movido e renomeado '2020_7158.JPG' para '2020_05_05_20_15_52.JPG' em 'imagens_mao\2020\feridas'
Created category folder: imagens_mao\2024\feridas
Movido e renomeado 'IMG_6199.JPEG' para '2024_10_02_13_05_17.JPEG' em 'imagens_mao\2024\feridas'
Processando categoria: flores
Created category folder: imagens_mao\2022\flores
Movido e renomeado '2022_11.JPEG' para '2022_03_21_14_19_48.JPEG' em 'imagens_mao\2022\flores'
Movido e renomeado

### Criar dicionário para usar em js com fotos

In [9]:
def generate_photos_dictionary(output_folder):
    """
    Gera um dicionário de caminhos de fotos organizados por ano e categoria.
    """
    photos = {}

    # Itera sobre cada pasta de ano (2019, 2020, etc.)
    for year_folder in os.listdir(output_folder):
        year_folder_path = os.path.join(output_folder, year_folder)

        # Verifica se é uma pasta de ano
        if os.path.isdir(year_folder_path):
            photos[year_folder] = {}  

            # Itera sobre cada pasta de categoria dentro do ano
            for category_folder in os.listdir(year_folder_path):
                category_folder_path = os.path.join(year_folder_path, category_folder)

                # Verifica se é uma pasta de categoria
                if os.path.isdir(category_folder_path):
                    # Inicializa uma lista para armazenar os caminhos das fotos da categoria
                    photos[year_folder][category_folder] = []

                    # Itera sobre cada arquivo de imagem dentro da categoria
                    for filename in os.listdir(category_folder_path):
                        # Verifica se é um arquivo de imagem pelos formatos comuns
                        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
                            # Caminho completo para a imagem
                            image_path = os.path.join("data\\",category_folder_path, filename)
                            # Adiciona o caminho da imagem à lista da categoria
                            photos[year_folder][category_folder].append(image_path)

    return photos


def save_photos_to_txt(photos, filename="photos.txt"):
    """
    Salva o dicionário de fotos em um arquivo .txt.
    """
    with open(filename, "w") as file:
        json.dump(photos, file, indent=4)
    print(f"Photos dictionary saved to {filename}")

In [10]:
photos_dict = generate_photos_dictionary(output_folder)
save_photos_to_txt(photos_dict)

Photos dictionary saved to photos.txt
